In [47]:
'''
prompt:
Escreva código python que lê todos os arquivos PDF da pasta data/exams e converte eles para texto.  A função parse_pdfs deve retornar uma lista onde cada elemento é o texto de cada PDF.
'''

'\nprompt:\nEscreva código python que lê todos os arquivos PDF da pasta data/exams e converte eles para texto.  A função parse_pdfs deve retornar uma lista onde cada elemento é o texto de cada PDF.\n'

In [48]:
!pip install PyPDF2




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\pedro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [49]:
import os
from PyPDF2 import PdfReader

def parse_pdfs(folder_path="data/exams"):
    textos = []
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            try:
                reader = PdfReader(pdf_path)
                texto = ""
                for page in reader.pages:
                    texto += page.extract_text() or ""
                textos.append(texto.strip())
            except Exception as e:
                print(f"Erro ao processar {filename}: {e}")
    
    return textos


In [50]:
pdf_textos = parse_pdfs()
for i, texto in enumerate(pdf_textos):
    print(f"\n--- PDF {i+1} ---\n")
    print(texto[:500])  # Mostra os primeiros 500 caracteres de cada PDF




--- PDF 1 ---

Quest ões comentadas O AB/FG V
Selecione a Matéria
Selecione o T ema
Exame O AB
FIL TRAR
Foram encontradas 3422  questões
Matéria: ÉTICA PR OFISSIONAL
Questão:  001
Prova: XV
Tema:  INCOMP ATIBILID ADE E IMPEDIMENT O
Abelardo é magistrado vinculad o ao Tribunal de Justiça do Estado K e requer licença
para tratamento de questões particulares, pelo prazo de três anos, o que foi deferido.
Como, antes de assumir o referido cargo, era advogado regularmente inscrito nos
quadros da OAB, requer o seu re

--- PDF 2 ---

Quest ões comentadas O AB/FG V
Selecione a Matéria
Selecione o T ema
Exame O AB
FIL TRAR
Foram encontradas 3422  questões
Matéria: ÉTICA PR OFISSIONAL
Questão:  091
Prova: XII
Tema:  INFRAÇÕES E SANÇÕES DISCIPLINARES
Fernanda, advogada regularmente inscrita nos quadros da OAB, atua, individualmente,
sem sócios, em seu escritório situado no centro da cidade “Z”, onde recebe os seus
clientes para atividades de assessoria e consultoria, atuando também no contencioso

In [51]:
'''
prompt:
Escreva código python que recebe uma string de texto e retorna uma lista de dicionários. Cada elemento da lista deve conter as chaves "materia", "questão", "prova" e "tema", em que o valor é o texto entre a chave e \n". Exemplo: \nMatéria: ÉTICA PR OFISSIONAL\nQuestão:  001\nProva: XV\nTema:  INCOMP ATIBILID ADE E IMPEDIMENT O\n

Depois, o "enunciado" está entre o que vem depois do Tema e A). Leia A), B), C) e D) como entradas distintas no dicionário.

Em cada alternativa, há o texto "Resposta errada" ou "Resposta correta". Cada alternativa deve ter 3 sub-chaves: o texto, um booleano que diz se a resposta é correta, e a a justificativa que vem a seguir.

'''

'\nprompt:\nEscreva código python que recebe uma string de texto e retorna uma lista de dicionários. Cada elemento da lista deve conter as chaves "materia", "questão", "prova" e "tema", em que o valor é o texto entre a chave e \n". Exemplo: \nMatéria: ÉTICA PR OFISSIONAL\nQuestão:  001\nProva: XV\nTema:  INCOMP ATIBILID ADE E IMPEDIMENT O\n\n\nDepois, o "enunciado" está entre o que vem depois do Tema e A). Leia A), B), C) e D) como entradas distintas no dicionário.\n\nEm cada alternativa, há o texto "Resposta errada" ou "Resposta correta". Cada alternativa deve ter 3 sub-chaves: o texto, um booleano que diz se a resposta é correta, e a a justificativa que vem a seguir.\n\n'

In [52]:
import re

import re

def parse_questoes(texto):
    padrao_bloco = re.compile(r'\nMatéria: (.*?)\nQuestão: (.*?)\nProva: (.*?)\nTema: (.*?)\n', re.DOTALL)
    blocos = list(padrao_bloco.finditer(texto))
    questoes = []

    for i, bloco in enumerate(blocos):
        materia = bloco.group(1).strip()
        questao = bloco.group(2).strip()
        prova = bloco.group(3).strip()
        tema = bloco.group(4).strip()

        # Determinar os limites do bloco de questão
        inicio_enunciado = bloco.end()
        fim_bloco = blocos[i + 1].start() if i + 1 < len(blocos) else len(texto)
        conteudo_questao = texto[inicio_enunciado:fim_bloco]

        # Separar enunciado e alternativas
        alternativas = re.split(r'\n([A-D]\))', conteudo_questao)
        enunciado = alternativas[0].strip()
        alternativas_dict = {}

        for j in range(1, len(alternativas) - 1, 2):
            letra = alternativas[j].strip(')')
            corpo = alternativas[j + 1].strip()

            # Extrair texto, resposta e justificativa
            match = re.match(r'^(.*?)(Resposta (correta|errada))(.*)$', corpo, re.DOTALL)
            if match:
                texto_alt = match.group(1).strip()
                correta = match.group(3).strip().lower() == 'correta'
                justificativa = match.group(4).strip()
            else:
                texto_alt = corpo
                correta = None
                justificativa = ''

            alternativas_dict[letra] = {
                'texto': texto_alt,
                'correta': correta,
                'justificativa': justificativa
            }

        questoes.append({
            'materia': materia,
            'questao': questao,
            'prova': prova,
            'tema': tema,
            'enunciado': enunciado,
            'alternativas': alternativas_dict
        })

    return questoes


### TESTES ####

questoes = parse_questoes(pdf_textos[0])

assert len(questoes) == 8

questao = questoes[0]

assert questao['prova'] == 'XV'
assert questao['questao'] == '001'

assert questao['enunciado'].startswith("Abelardo")

assert questao['alternativas']['B']['correta'] == False
assert questao['alternativas']['C']['correta'] == True
assert questao['alternativas']['D']['correta'] == False



In [ ]:
'''
prompt:
Tenho uma lista de dicionários. Cada dicionário tem uma chave "prova", "materia", "questao" e "enunciado". 
Gere  código que produz um pandas dataframe salva em data/oab.csv
'''

In [59]:
import pandas as pd

def salvar_provas(lista_de_questoes, caminho_saida="data/oab.csv"):
    # Cria o DataFrame
    df = pd.DataFrame(lista_de_questoes)

    # Garante a ordem das colunas
    colunas_desejadas = ["prova", "materia", "questao", "enunciado"]
    df = df[colunas_desejadas]

    # Salva como CSV
    df.to_csv(caminho_saida, index=False, encoding="utf-8")
    print(f"Arquivo salvo em {caminho_saida}")

# Supondo que cada elemento de pdf_textos seja uma lista de dicionários
todas_questoes = []

for questoes_originais in pdf_textos:
    questoes = parse_questoes(questoes_originais)
    todas_questoes.extend(questoes)  # Adiciona todas as questões ao acumulador

salvar_provas(todas_questoes)


Arquivo salvo em data/oab.csv
